# Data Preprocessing

In [2]:
from pyhealth.datasets import MIMIC3Dataset

dataset = MIMIC3Dataset(
    root='data/',
    tables=["DIAGNOSES_ICD","PROCEDURES_ICD"],
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})}
)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
INIZIO PREPROCESSING


finish basic patient information parsing : 32.21064639091492s
finish parsing DIAGNOSES_ICD : 28.0324969291687s
finish parsing PROCEDURES_ICD : 20.501681327819824s


Mapping codes: 100%|██████████| 46520/46520 [00:00<00:00, 54409.77it/s]


In [3]:
dataset.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PROCEDURES_ICD: 4.0711



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

In [4]:
# get patient dictionary
patient_dict = dataset.patients
print(list(patient_dict.keys())[:10])

['10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005', '10006']


In [5]:
# get the "10" patient
patient = patient_dict["10"]
patient.gender, patient.birth_datetime, patient.ethnicity, patient.death_datetime, patient.visits
print(patient.visits)

OrderedDict([('184167', Visit 184167 from patient 10 with 8 events from tables ['DIAGNOSES_ICD', 'PROCEDURES_ICD'])])


In [6]:
# get the visit list of this patient
visit_dict = patient.visits
print (list(visit_dict.keys()))

['184167']


In [7]:
# get the first visit
visit = visit_dict['184167']
visit.encounter_time, visit.available_tables, visit.num_events, visit.event_list_dict

(datetime.datetime(2103, 6, 28, 11, 36),
 ['DIAGNOSES_ICD', 'PROCEDURES_ICD'],
 8,
 {'DIAGNOSES_ICD': [Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
   Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
   Event with ICD9CM code V290 from table DIAGNOSES_ICD],
  'PROCEDURES_ICD': [Event with ICD9PROC code 9983 from table PROCEDURES_ICD,
   Event with ICD9PROC code 9915 from table PROCEDURES_ICD,
   Event with ICD9PROC code 966 from table PROCEDURES_ICD]})

In [10]:
visit.get_event_list('DIAGNOSES_ICD')

[Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
 Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
 Event with ICD9CM code V290 from table DIAGNOSES_ICD]

In [11]:
visit.get_code_list('DIAGNOSES_ICD')

['V3000', '7742', '76525', '76515', 'V290']

In [12]:
import pandas as pd

table = "NOTEEVENTS"

# read table
df = pd.read_csv(
    "data/NOTEEVENTS.csv",
    dtype={"SUBJECT_ID": str, "HADM_ID": str, "CATEGORY": str,"CHARTDATE": str, "TEXT": str},
    parse_dates=["CHARTDATE"],
    infer_datetime_format=True
)


C:\Users\giuse\AppData\Local\Temp\ipykernel_12636\2530625872.py:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [13]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [14]:
# filter for 'DISCHARGE SUMMARY' in the 'CATEGORY' column
df = df[df['CATEGORY'] == 'Discharge summary']
# drop records of the other patients
df = df[df["SUBJECT_ID"].isin(patient_dict.keys())]
# drop rows with missing values
df = df.dropna(subset=["SUBJECT_ID", "HADM_ID", "TEXT"])
# group by patient and visit
group_df = df.groupby("SUBJECT_ID")

In [15]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
60411,59608,93924,135672,2104-06-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18321**],[**Known fir..."
60412,59609,17782,155710,2112-03-31,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 2706**], [**Known fir..."
60413,59621,96333,183951,2197-06-18,NaN,NaN,Discharge summary,Addendum,NaN,NaN,Name: [**Known lastname 18357**]-[**Known las...
60414,59622,66717,169165,2129-08-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname **],[**Known firstnam..."


In [16]:
# Define the ranges for subject IDs
ranges = [(10000, 20000), (20000, 30000), (30000, 40000)]

# Filter 5 patients from each range
selected_patients = []
for start, end in ranges:
    patients_in_range = df[(df['SUBJECT_ID'].astype(int) > start) & (df['SUBJECT_ID'].astype(int) <= end)]
    selected_patients.extend(patients_in_range.sample(5).to_dict('records'))

# Convert the list of dicts to a dataframe
selected_patients_df = pd.DataFrame(selected_patients)



In [17]:
selected_patients_list = selected_patients_df['SUBJECT_ID'].tolist()
selected_patients_list

['13631',
 '12285',
 '15923',
 '19593',
 '12140',
 '20629',
 '25333',
 '24444',
 '24398',
 '23460',
 '30225',
 '32247',
 '31526',
 '30299',
 '32040']

In [18]:
filtered_noteevents_df = df[df['SUBJECT_ID'].isin(selected_patients_list)]

In [19]:
filtered_noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
4738,4744,32040,192895,2130-05-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2130-5-12**] ...
5315,5087,12140,183207,2144-06-02,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2144-5-29**] Dischar...
5316,5088,12140,188829,2151-04-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-3-26**] ...
10797,7974,24444,165264,2167-07-06,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2167-6-30**] ...
10798,7975,24444,195909,2168-07-22,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2168-7-21**] ...
10799,7976,24444,195909,2168-08-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2168-7-22**] ...
10800,7977,24444,108879,2168-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2168-8-16**] ...
12506,14870,19593,164641,2198-09-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2198-8-27**] Dischar...
14023,11494,15923,116118,2165-03-07,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2165-3-1**] Discharg...
14173,15061,13631,134393,2139-08-05,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2139-7-30**] Dischar...


In [ ]:
#unique_subject_ids = noteevents_df['SUBJECT_ID'].unique()[:10]
#filtered_noteevents_df = noteevents_df[noteevents_df['SUBJECT_ID'].isin(unique_subject_ids)]

In [20]:
from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT

c:\Users\giuse\Desktop\MSc-Thesis-Project\proj_venv\lib\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [21]:
DATA_DIR = "./data_umls/"
vocab_path = DATA_DIR + "vocab.dat"
cdb_path = DATA_DIR + "cdb.dat"

In [22]:
# Create and load the CDB (Concept Database)
cdb = CDB.load(cdb_path)

# Create and load the Vocabulary
vocab = Vocab.load(vocab_path)

# Load model for MetaAnnotations (optional)
mc_status = MetaCAT.load("./data_umls/meta_Status/")

In [23]:
# Create CAT - the main class from medcat used for concept annotation
cat = CAT(cdb=cdb, config=cdb.config, vocab=vocab, meta_cats=[mc_status])

c:\Users\giuse\Desktop\MSc-Thesis-Project\proj_venv\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [24]:
# Detect only the following entity:
    # bpoc|T023|Body Part, Organ, or Organ Component
    # lbtr|T034|Laboratory or Test Result
    # patf|T046|Pathologic Function
    # dsyn|T047|Disease or Syndrome
    # mobd|T048|Mental or Behavioral Dysfunction
    # lbpr|T059|Laboratory Procedure
    # diap|T060|Diagnostic Procedure
    # topp|T061|Therapeutic or Preventive Procedure
    # phsu|T121|Pharmacologic Substance ---> SE LO AGGIUNGO MI METTE ANCHE ALTRA ROBA TIPO VITAMINE, ENZIMI, ETC.
    # sosy|T184|Sign or Symptom

type_ids_filter = ['T047','T048','T184']
cui_filters = set()
for type_ids in type_ids_filter:
    cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters

In [25]:
color_dict = {
    'Body Part, Organ, or Organ Component': '#FFCE80',  # Arancione pastello
    'Laboratory or Test Result': '#FFF9C4',             # Giallo chiaro
    'Disease or Syndrome': '#B5EAD7',                   # Verde acqua chiaro
    'Mental or Behavioral Dysfunction': '#F0B2FF',      # Viola pastello
    'Laboratory Procedure': '#D0D9FF',                  # Blu chiaro
    'Diagnostic Procedure': '#FFD9EC',                  # Rosa chiaro
    'Therapeutic or Preventive Procedure': '#C4FFFF',   # Azzurro chiaro
    'Pharmacologic Substance': '#FFDAB9',               # Pesca pastello
    'Sign or Symptom': '#FFC4F3',                       # Magenta chiaro
    'Pathologic Function': '#B5EAD7'                    # Verde acqua chiaro
}

In [ ]:
entities_lemma = cat.get_entities(filtered_noteevents_df['TEXT'][1286])

In [ ]:
entities_lemma

In [ ]:
extracted_data = []
# Extract 'pretty_name' and 'types' values from the dictionary
for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        if value['types'][0] == 'Pharmacologic Substance':
            extracted_data.append((value['source_value'], value['types'][0], value['icd10']))
        else:
            extracted_data.append((value['pretty_name'], value['types'][0], value['icd10']))

# Create a dataframe
df_extra = pd.DataFrame(extracted_data, columns=['Nome', 'Entità', 'ICD10'])


In [ ]:
from spacy import displacy

doc = {"text": df['TEXT'][1286], "ents": [], "title": None}

for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        ent = {
            'start': value['start'],
            'end': value['end'],
            'label': value['types'][0]
        }
        doc['ents'].append(ent)

colors = color_dict
options = {"ents": list(colors.keys()), "colors": colors}

# Call spacy.displacy.render() to visualize the entities
displacy.render(doc, style='ent', options=options, jupyter=True, manual=True)

In [ ]:
extracted_data